## What's `Dependency Injector` ?

`Denpendency Injector` is a dependency injection framework for Python.
It helps implementing the DI principle.

Key features of the `Dependency Injector`.

- **Providers** : Provides some providers that helps assembling your objects. 
  - `Factory`,
  - `Singleton`,
  - `Callable`,
  - `Coroutine`,
  - `Object`,
  - `List`,
  - `Dict`,
  - `Configuration`,
  - `Resource`,
  - `Dependency`,
  - `Selector`


- **Overriding** : Can override any provider by another provider on the fly. This helps in testing and configuring dev / stage environment to replace API clients with stubs etc.
- **Configuration** : Reads configuration from yaml, ini files, `pydantic` settings, environment variables, and dictionaries. See `Configuration provider`.
- **Containers** : Provides declarative and dynamic containers. See `Containers`.

- **Resource** : Helps with initialization and configuring of logging, event  loop, thread or process pool, etc. Can be used for per-function execution scope in tandem with wiring. See `Resource provider`.
- **Wiring** : Injects dependencies into functions and methods. Helps integrating with other frameworks: `Django`, `FastAPI`. See `Wiring`
- **Asynchronous** : Supports asynchronous injections.
- **Typing** : Provides typing stubs, `mypy`-friendly.
- **Performance** : Fast. Written in Cython.
- **Maturity** : Mature and production-ready. Well-tested, documented and supported.

In [1]:
from dependency_injector import containers, providers
from dependency_injector.wiring import Provide, inject

In [2]:
class ApiClient():
    pass

class Service():
    pass

class Container(containers.DeclarativeContainer):
    config = providers.Configuration()
    
    api_client = providers.Singleton(
        ApiClient,
        api_key=config.api_key,
        timeout=config.timeout.as_int(),
    )
    service = providers.Factory(
        Service,
        api_clinet=api_client,
    )
    
class ServiceMock():
    pass

In [3]:
@inject
def main(service: Service=Provide[Container.service]):
    pass

In [4]:
## ---------------// entry point
container = Container()
container.config.api_key.from_env("API_KEY")
container.config.timeout.from_env("TIMEOUT")


In [5]:
str(container.config.api_key)

"<dependency_injector.providers.ConfigurationOption('config.api_key') at 0x22ff67930b0>"

In [6]:
container.wire(modules=[__name__])

with container.api_client.override(ServiceMock()):
    main()

TypeError: Service() takes no arguments

In [ ]:
dir(container.config)

In [7]:
import sys

In [8]:
class School:
    def __init__(self, name:str, city:str, board:str):
        self.schoolname = name
        self.city = city
        self.board = board
    
    def get_schoolname(self) -> None:
        print(f'school name is {self.schoolname}')
        


In [19]:
class Student():
    def __init__(self, name:str, age:int, grade:int, school:School):
        self.name = name
        self.age = age
        self.grade = grade
        self.school = school

    def get_student(self):
        student_detail = {
            "name": self.name,
            "age": self.age,
            "grade": self.grade,
            "school": self.school.schoolname,
            "board": self.school.board,
            "city": self.school.city
        }
        print(student_detail)

In [20]:
class Container(containers.DeclarativeContainer):
    school = providers.Factory(
        School,
        "Kendriya Vidhyala",
        "C.V.S.E",
        "Bina"
    )
    
    student = providers.Factory(
        Student,
        "Siddhartha",
        28,
        12,
        school
    )

In [21]:
container = Container()


In [22]:
__name__

'__main__'

In [24]:
sys.modules[__name__]

<module '__main__'>

In [25]:
container.wire(modules=[sys.modules[__name__]])


In [26]:
student = Container.student()
student.get_student()

{'name': 'Siddhartha', 'age': 28, 'grade': 12, 'school': 'Kendriya Vidhyala', 'board': 'Bina', 'city': 'C.V.S.E'}


In [27]:
school = Container.school()

In [28]:
school.get_schoolname()

school name is Kendriya Vidhyala


In [29]:
class MockContainer(containers.DeclarativeContainer):
    school = providers.Factory(
        School,
        "Senda",
        "Hights",
        "2002"
    )
    
    student = providers.Factory(
        Student,
        "Hello",
        20,
        12,
        school
    )

In [30]:
container = MockContainer()

In [31]:
container.wire(modules=[sys.modules[__name__]])

In [33]:
student = MockContainer.student()
student.get_student()

{'name': 'Hello', 'age': 20, 'grade': 12, 'school': 'Senda', 'board': '2002', 'city': 'Hights'}
